### SciPy versus NumPy

In [ ]:
# Import numpy symbols to scipy namespace
from numpy import *
from numpy.random import rand, randn
from numpy.fft import fft, ifft
from numpy.lib.scimath import *

In [ ]:
import numpy as np
a = np.identity(3)

### Statistics